In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys
path = os.getcwd()
while not path.endswith('research'):    
    path = os.path.abspath(os.path.join(path, '../'))
sys.path.append(path)

***

In [3]:
# helpful packages
import os, sys
import pandas as pd
import numpy as np
from datetime import datetime, date
import matplotlib.pyplot as plt
from datetime import timedelta

In [4]:
#a number of gas analytics utilities:
from gmt.fom.gas.shared.utils.common import pa
from gmt.fom.gas.shared.utils.dframe import ddff

In [5]:
#gas aligned mongo db
from gmt.fom.gas.shared.data_sources.mongo import GasMongo

#  Rebuild_ENTSOG_capacity.py

In [6]:
# %load Rebuild_ENTSOG_capacity.py
import pandas as pd
import datetime
import numpy as np

#----------------Dates used in API URL ----------------------------------------------------------------------
today = datetime.date.today()
yesterday = today - datetime.timedelta(days = 1)
tomorrow = today + datetime.timedelta(days = 665)
#------------------------------------------------------------------------------------------------------------

#----------------Further variables to change for API request ------------------------------------------------
fromDate = yesterday.strftime('%Y-%m-%d')
toDate = tomorrow.strftime('%Y-%m-%d')
#toDate = datetime.datetime.today().strftime('%Y-%m-%d')
#------------------------------------------------------------------------------------------------------------

In [7]:
#============================================================
#THIS QUERIES ENTSOG AND RETURNS REONOMINATIONS FOR ALL OPERATORS

url = (
       "https://transparency.entsog.eu/api/v1/operationalData.csv?&"+
       "delimiter=comma&" +
       "indicator=" +
       "Firm%20Technical," +
       "Firm%20Booked," +
       "Firm%20Available," +
       "Interruptible%20Total," +
       "Interruptible%20Booked" +
       "Interruptible%20Available" +
       "&periodType=day&timezone=CET&limit=-1&dataset=1" 
       +"&from="+ 
       fromDate +
       "&to="+ 
       toDate
       )


df = pd.read_csv(url)

C:\Users\ashubert\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (15,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
columns = ['id', 'indicator', 'periodType', 'periodFrom', 'periodTo',
       'operatorKey', 'tsoEicCode', 'operatorLabel', 'pointKey', 'pointLabel',
       'tsoItemIdentifier', 'directionKey', 'unit', 'lastUpdateDateTime',
       'value']
df = df[columns]
#These need to be added ['fromCountry', 'fromCountryCode', 'toCountry', 'fromSystem', 'toCountryCode', 'toSystem']

In [9]:
#----Converts the columns into datetime & set the index
df[['periodFrom','periodTo','lastUpdateDateTime']] = df[['periodFrom','periodTo','lastUpdateDateTime']].apply(pd.to_datetime)
#df[['value']]= df[['value']] * 9.09e-8
df[['unit']]= 'GWh/d'
df[['value']]= df[['value']] * 1.0e-7
df.reset_index(drop=True, inplace=True)
df = df.sort_values(['tsoItemIdentifier','directionKey']).copy()

In [10]:
#====================================
#Creates a list of all the operator keys from the Full Data so that we search for the to and from countries below
TotalOperatorKeys = df['operatorKey'].unique()
TotalOperatorKeys = TotalOperatorKeys.tolist()

#==================================
#Looks up all the Countries each connection if flow from.
CountryconnectionID = pd.DataFrame()

for url in TotalOperatorKeys:
    try:
        url = "https://transparency.entsog.eu/api/v1/Interconnections.csv?&fromOperatorKey=" + url
        df1=pd.read_csv(url)
        CountryconnectionID = CountryconnectionID.append(df1)
    except:
        pass

Columns = ['fromSystemLabel', 'fromCountryKey', 'fromCountryLabel',
       'fromOperatorKey',  'fromTsoItemIdentifier',
       'toSystemLabel', 'toCountryKey', 'toCountryLabel']

CountryconnectionID = CountryconnectionID[Columns]

CountryconnectionID.rename(columns={'fromTsoItemIdentifier':'tsoItemIdentifier',
                                    'fromOperatorKey':'operatorKey'}, inplace=True)
#=====================================

In [11]:
FromCountryData = pd.merge(df, CountryconnectionID, on=['operatorKey','tsoItemIdentifier'], how='outer')
ToCountryData = FromCountryData[FromCountryData['fromCountryLabel'].isnull()]

FromCountryData = FromCountryData.dropna()
ToCountryData = ToCountryData[ToCountryData['tsoEicCode'].notna()]

FromCountryData.rename(columns={'fromCountryKey':'toCountryCode',
                      'fromSystemLabel':'toSystem',
                      'fromCountryLabel':'toCountry',
                      'toCountryLabel':'fromCountry',
                      'toSystemLabel':'fromSystem',
                      'toCountryKey':'fromCountryCode'}, inplace=True)
#==========================================

In [12]:
columns = ['id', 'indicator', 'periodType', 'periodFrom', 'periodTo',
       'operatorKey', 'tsoEicCode', 'operatorLabel', 'pointKey', 'pointLabel',
       'tsoItemIdentifier', 'directionKey', 'unit', 'lastUpdateDateTime',
       'value']

ToCountryData = ToCountryData[columns]

In [13]:
#==========================================
#Looks up all the connections that countries are flowing to

Other_CountryconnectionID = pd.DataFrame()

for url in TotalOperatorKeys:
    try:
        url = "https://transparency.entsog.eu/api/v1/Interconnections.csv?&toOperatorKey=" + url
        df1=pd.read_csv(url)
        
        Other_CountryconnectionID = Other_CountryconnectionID.append(df1)
    except:
        pass
    
columns = ['fromSystemLabel', 'fromCountryKey', 'fromCountryLabel',
       'toSystemLabel', 'toCountryKey', 'toCountryLabel', 'toOperatorKey',
       'toTsoItemIdentifier']

Other_CountryconnectionID = Other_CountryconnectionID[columns]

#Renames the columns so that the different dataframes can me joined together  
Other_CountryconnectionID.rename(columns={'toTsoItemIdentifier':'tsoItemIdentifier',
                                          'toOperatorKey':'operatorKey'}, inplace=True)

#==========================================
#Joins the dataframes on the columns tsoItemIdentifer and operatorKey
OtherCountryData = pd.merge(ToCountryData, Other_CountryconnectionID, on=['operatorKey','tsoItemIdentifier'], how='outer')
#Keeps only the rows with data in the tsoEicCode column
OtherCountryData = OtherCountryData[OtherCountryData['tsoEicCode'].notna()]
OtherCountryData = OtherCountryData[OtherCountryData['pointLabel'].notna()]
OtherCountryData = OtherCountryData.dropna()

OtherCountryData.rename(columns={'fromSystemLabel':'fromSystem',
                                 'fromCountryKey':'fromCountryCode',
                                 'fromCountryLabel':'fromCountry',
                                 'toSystemLabel':'toSystem',
                                 'toCountryKey':'toCountryCode',
                                 'toCountryLabel':'toCountry'}, inplace=True)
#========================================================

In [14]:
#Combines the two complete dataframes together
Final = OtherCountryData.append([FromCountryData])
#Removes the foregin letters
Final['pointLabel'] = Final['pointLabel'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
Final['operatorLabel'] = Final['operatorLabel'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
Final['fromSystem'] = Final['fromSystem'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
Final['toSystem'] = Final['toSystem'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

Final['value'] = pd.to_numeric(Final['value'])


Final2 = Final.copy()

C:\Users\ashubert\Anaconda3\envs\short_term_power_burn\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [15]:
#=======================================================
#===========================================================
Final2.set_index('pointLabel', inplace=True)
Final2['row'] = range(len(Final2))


starts = Final2[['periodFrom', 'value', 'indicator', 'id', 'periodType',
       'operatorKey', 'tsoEicCode', 'operatorLabel', 'pointKey',
       'tsoItemIdentifier', 'directionKey', 'unit', 'lastUpdateDateTime',
       'fromSystem',
       'fromCountryCode', 'fromCountry', 'toSystem', 'toCountryCode',
       'toCountry' ,'row']].rename(columns={'periodFrom': 'Date'})

ends = Final2[['periodTo', 'value', 'indicator','id', 'periodType',
       'operatorKey', 'tsoEicCode', 'operatorLabel', 'pointKey',
       'tsoItemIdentifier', 'directionKey', 'unit', 'lastUpdateDateTime',
       'fromSystem',
       'fromCountryCode', 'fromCountry', 'toSystem', 'toCountryCode',
       'toCountry','row']].rename(columns={'periodTo':'Date'})

#================================================================
df_decomp = pd.concat([starts, ends]).drop_duplicates()
df_decomp = df_decomp.set_index('row', append=True)
df_decomp.sort_index()

df_decomp = df_decomp.groupby(level=[0,1]).apply(lambda x: x.set_index('Date').resample('D').fillna(method='pad'))
df_decomp = df_decomp.reset_index(level=1, drop=True)
df_decomp = df_decomp.reset_index()
df_decomp = df_decomp.drop_duplicates()
df_decomp = df_decomp.dropna()
df_decomp = df_decomp.rename(columns={'Date':'periodFrom'})


df_decomp["periodTo"] =  df_decomp["periodFrom"] + timedelta(days=1)
#===================================================================

In [17]:
df_decomp.shape

(2462127, 21)

# GAS RESEARCH

In [6]:
from gmt.fom.gas.shared.data_sources.entsog_capacity import EntsogCapacityDataScrapper, MongoAdaptor

In [7]:
dataScapper = EntsogCapacityDataScrapper()

2021-03-18 09:52:48,957: c:\git\gmt-fom-gas-research\gmt\fom\gas\shared\data_sources\mongo.py: 25: DEBUG: Connected successfully.


In [20]:
dataScapper.query_data()

C:\Users\ashubert\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (15,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [21]:
dataScapper.filter_columns()

In [22]:
dataScapper.convert_columns()

In [23]:
dataScapper.country_connection_id()

In [24]:
dataScapper.from_country_data()

In [25]:
dataScapper.other_countries_connection_id()

In [26]:
dataScapper.combine()

In [28]:
dataScapper.df_decomp.shape

(2462127, 21)

In [31]:
ddff(dataScapper.df_decomp.head())

pointLabel periodFrom  \
1  's Gravenvoeren Dilsen (BE) // 's Gravenvoeren/Obbicht (NL) 2021-03-17   
2  's Gravenvoeren Dilsen (BE) // 's Gravenvoeren/Obbicht (NL) 2021-03-18   
3  's Gravenvoeren Dilsen (BE) // 's Gravenvoeren/Obbicht (NL) 2021-03-19   
4  's Gravenvoeren Dilsen (BE) // 's Gravenvoeren/Obbicht (NL) 2021-03-20   
5  's Gravenvoeren Dilsen (BE) // 's Gravenvoeren/Obbicht (NL) 2021-03-21   

   value       indicator  \
1    0.0  Firm Technical   
2    0.0  Firm Technical   
3    0.0  Firm Technical   
4    0.0  Firm Technical   
5    0.0  Firm Technical   

                                                                     id  \
1  1Firm TechnicaldayNL-TSO-0001ITP-00258entrykWh/d2014-01-012051-02-25   
2  1Firm TechnicaldayNL-TSO-0001ITP-00258entrykWh/d2014-01-012051-02-25   
3  1Firm TechnicaldayNL-TSO-0001ITP-00258entrykWh/d2014-01-012051-02-25   
4  1Firm TechnicaldayNL-TSO-0001ITP-00258entrykWh/d2014-01-012051-02-25   
5  1Firm TechnicaldayNL-TSO-0001ITP-00258entrykWh/d2014-01-012051-02-25   

  periodType  operatorKey        tsoEicCode operatorLabel   pointKey  \
1        day  NL-TSO-0001  21X-NL-A-A0A0A-Z           GTS  ITP-00258   
2        day  NL-TSO-0001  21X-NL-A-A0A0A-Z           GTS  ITP-00258   
3        day  NL-TSO-0001  21X-NL-A-A0A0A-Z           GTS  ITP-00258   
4        day  NL-TSO-0001  21X-NL-A-A0A0A-Z           GTS  ITP-00258   
5        day  NL-TSO-0001  21X-NL-A-A0A0A-Z           GTS  ITP-00258   

  tsoItemIdentifier directionKey   unit  lastUpdateDateTime fromSystem  \
1  21Z000000000169F        entry  GWh/d 2020-02-26 16:08:00      BeLux   
2  21Z000000000169F        entry  GWh/d 2020-02-26 16:08:00      BeLux   
3  21Z000000000169F        entry  GWh/d 2020-02-26 16:08:00      BeLux   
4  21Z000000000169F        entry  GWh/d 2020-02-26 16:08:00      BeLux   
5  21Z000000000169F        entry  GWh/d 2020-02-26 16:08:00      BeLux   

  fromCountryCode fromCountry     toSystem toCountryCode    toCountry  \
1              BE     Belgium  Netherlands            NL  Netherlands   
2              BE     Belgium  Netherlands            NL  Netherlands   
3              BE     Belgium  Netherlands            NL  Netherlands   
4              BE     Belgium  Netherlands            NL  Netherlands   
5              BE     Belgium  Netherlands            NL  Netherlands   

    periodTo  
1 2021-03-18  
2 2021-03-19  
3 2021-03-20  
4 2021-03-21  
5 2021-03-22

In [32]:
dataScapper.df_decomp['run_day'] = dataScapper.run_day

In [33]:
from gmt.fom.gas.shared.data_sources.mongo import GasMongo

In [35]:
mongo = GasMongo()
mongo.choose_db(MongoAdaptor.GAS_MODELS_DB)

2021-03-17 10:55:47,151: c:\git\gmt-fom-gas-research\gmt\fom\gas\shared\data_sources\mongo.py: 25: DEBUG: Connected successfully.


In [36]:
mongo.choose_collection(MongoAdaptor.ENTSOG_CAPACITY_COLLECTION)

2021-03-17 10:55:56,736: c:\git\gmt-fom-gas-research\gmt\fom\gas\shared\data_sources\mongo.py: 48: INFO: Choosing collection entsog_capacity.


In [39]:
t = dataScapper.run_day

In [40]:
mongo.collection.delete_many(
    {EntsogCapacityDataScrapper.RUN_DAY: pd.to_datetime(t)})

In [54]:
s = dataScapper.df_decomp.dtypes
s.where(s=='datetime64[ns]').dropna().index.tolist()

['periodFrom', 'lastUpdateDateTime', 'periodTo']

In [52]:
type(dataScapper.df_decomp['run_day']

datetime.date

In [55]:
for c in ['periodFrom', 'lastUpdateDateTime', 'periodTo', 'run_day']:
    dataScapper.df_decomp[c] = pd.to_datetime(dataScapper.df_decomp[c])

In [ ]:
mongo.pandas_to_mongo(dataScapper.df_decomp)

2021-03-17 11:15:11,519: c:\git\gmt-fom-gas-research\gmt\fom\gas\shared\data_sources\mongo.py: 99: DEBUG: Saving 2462127 rows to MongoDb.


# Retrieve entsog data

In [ ]:
from gmt.fom.gas.shared.data_sources.entsog_capacity import EntsogCapacityDataScrapper, MongoAdaptor

In [12]:
myData = MongoAdaptor()

2021-03-18 09:54:52,849: c:\git\gmt-fom-gas-research\gmt\fom\gas\shared\data_sources\mongo.py: 25: DEBUG: Connected successfully.


In [13]:
t_range = myData.cached_t_range; t_range

[datetime.date(2021, 3, 17), datetime.date(2021, 3, 18)]

In [ ]:
# for a continium of days:

t0 = pd.to_datetime(date(2021,3,17))
t1 = pd.to_datetime(date(2021,3,18))

#  t0 <= T <= t1
query = {'run_day':{"$gte": t0, "$lte": t1}}

entsog = EntsogCapacityDataScrapper.retrieve(query=query)
entsog.shape

2021-03-18 09:59:01,511: c:\git\gmt-fom-gas-research\gmt\fom\gas\shared\data_sources\entsog_capacity.py: 265: INFO: Retrieving....
2021-03-18 09:59:02,278: c:\git\gmt-fom-gas-research\gmt\fom\gas\shared\data_sources\mongo.py: 25: DEBUG: Connected successfully.


In [ ]:
# for specific list of days (the days must be already cached):
# to check already cached days, use NetBackModel.get_cached_days() from above:

t1 = date(2021, 3, 17)
t2 = date(2018, 3, 18)


entsog = EntsogCapacityDataScrapper.retrieve([t1,t2])
entsog.shape